# PyTorch `tensor.backward()` fucntion
<font color="steelblue" size="4">

1. This post examines some `tensor.backward()` function examples about the `autograd (Automatic Differentiation)` package of PyTorch.
2. As you already know, if you want to compute all the derivatives of a tensor, you can call `backward()` on it. (`tensor.backward()`)
3. The `torch.tensor.backward()` relies on the autograd function `torch.autograd.backward()` that computes the `sum of gradients (without returning it) of given tensors with respect to the graph leaves`.

</font>

# 1. A first Example

## 1.1. Example 的描述
<font color="steelblue" size="4">

1. Give a matrix $x$,
$$\begin{aligned}
x :=
\begin{bmatrix}
x_1 & x_2 \\
x_3 & x_4
\end{bmatrix} = 
\begin{bmatrix}
1 & 1 \\
1 & 1
\end{bmatrix}
\end{aligned}$$
2. and another matrix y is defined as
$$\begin{aligned}
y := 
x + 2 = 
\begin{bmatrix}
x_1 + 2  &  x_2 + 2 \\
x_3 + 2  &  x_4 + 2
\end{bmatrix} = 
\begin{bmatrix}
3 & 3 \\
3 & 3 
\end{bmatrix}
\end{aligned}$$
3. We define z as:
    - <font color="red">Note: `*` -- entry-wise multiplication</font>
$$\begin{aligned}
z = y * y * 3 = 
3 * 
\begin{bmatrix}
y_1 & y_2 \\ 
y_3 & y_4
\end{bmatrix} * 
\begin{bmatrix}
y_1 & y_2 \\ 
y_3 & y_4
\end{bmatrix} = 
\begin{bmatrix}
3y_1^2 & 3y_2^2 \\ 
3y_3^2 & 3y_4^2
\end{bmatrix}
\end{aligned}$$
4. Finally, we define out as:
$$\begin{aligned}
out = \frac{1}{4}(3y_1^2 + 3y_2^2 + 3y_3^2 + 3y_4^2)
\end{aligned}$$
5. The value of derivative of out with respect with $x_2$
$$\begin{aligned}
\frac{\partial out}{\partial x_2}
&= \frac{\partial}{\partial x_2} \left( \frac{1}{4}(3y_1^2+3y_2^2+3y_3^2+3y_4^2) \right) \\
&= 0 + \frac{3}{4}\frac{\partial}{\partial x_2}y_2^2 + 0 + 0 \\
&= \frac{3}{4}\frac{\partial}{\partial x_2}(x_2 + 2)^2 \\
&= \frac{3}{2}(x_2 + 2)
\end{aligned}$$

</font>


<font color="coral" size="4">

Note
----
1. $out$ contains `a single real value`(`scaler`). This value is the result of `scalar function`(In the case, the `mean` function).

</font>

## 1.2. Code for example

<font color="coral" size="4">

Note
----
1. The `grad` attribute of `x` is None by default and becomes a tensor the first time a call to `out.backward()` computes gradients for self($\frac{\partial out}{\partial x}$).
2. The `grad` attribute will then contain the gradients computed and future calls to `backward()` will accumulate (add) gradients into it. 

</font>

In [9]:
import torch

### Part I. device
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

print(device)

### Part II. example code
## Step 1. define how to calculate
x = torch.ones(
            (2, 2),
            device=device,
            requires_grad=True,
            dtype=torch.float32,
            )

# y.grad_fn = AddBackward
y = x + 2
# z.grad_fn = MultiBackward
z = y * y  * 3  # element-wise multiplcation
# out.grad_fn = MeanBackward
output = z.mean()
print("out (a scaler) = ", output.item())

## Step 2. calculate the `derivatives of out with respect to x`: dout/dx
output.backward()
print("dout/dx = ", x.grad)

cpu
out (a scaler) =  27.0
dout/dx =  tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


## 1.3. `x.grad` will accumulate if not `optimizer.zero_grad()` in training loop
<font color="coral" size="4">

Note
----
1. The `grad` attribute will then contain the gradients computed and future calls to `backward()` will accumulate (add) gradients into it. 

</font>

In [20]:
# Step 1. init torch.tensor x
x = torch.ones(
            (2, 2),
            device=device,
            requires_grad=True,
            dtype=torch.float32,
            )

def calculate_output(x: torch.tensor):
    y = x + 2
    z = y * y * 3   # element-wise multiplication
    output = z.mean()
    output.backward()


# Step 2. first time
calculate_output(x=x)
print("First:\n\tdout/dx = ", x.grad)
print("")


# Step 3. second time
calculate_output(x=x)
print("Second:\n\tdout/dx = ", x.grad)

First:
	dout/dx =  tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])

Second:
	dout/dx =  tensor([[9., 9.],
        [9., 9.]])
